In [1]:
import math
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
from numpy.linalg import inv
from scipy.stats import ncx2
from scipy.optimize import fsolve
from scipy.special import comb
from scipy.stats import binom

# Black-Scholes

In [2]:
def BS_price(S, K, T, r, sigma, div, x):
    # x=1 means call option, x=0 means put option
    d1 = (np.log(S/K) + (r + (0.5*sigma**2) - div)*T)/(sigma*np.sqrt(T))
    d2 = (np.log(S/K) + (r - (0.5*sigma**2) - div)*T)/(sigma*np.sqrt(T))
    p = (2*x-1)*S*np.exp((-1)*div*T)*norm.cdf((2*x-1)*d1, 0.0, 1.0) - (2*x-1)*K*np.exp(-r*T)*norm.cdf((2*x-1)*d2, 0.0, 1.0)
    return p

# Halton's Low-Discrepancy Sequences

In [3]:
def van_der_corput(n_sample, base=2):
    sequence = []
    for i in range(n_sample):
        n_th_number, denom = 0., 1.
        while i > 0:
            i, remainder = divmod(i, base)
            denom *= base
            n_th_number += remainder / denom
        sequence.append(n_th_number)
    return sequence

In [4]:
def halton(dim, n_sample, base):
    sample = [van_der_corput(n_sample + 1, dim) for dim in base]
    sample = np.stack(sample, axis=-1)[1:]
    return sample

In [5]:
# Use Halton’s Low-Discrepancy Sequences to price European Call options.
def HaltonEurCall(S0,X,T,r,sigma,N,b1,b2):
    
    H1 = halton(2, N, [b1,b2])[:,0]
    H2 = halton(2, N, [b1,b2])[:,1]
    
    # Box- Muller Method    
    z1 = []
    z2 = []
    payoff = []
    for i in range(0,N):
        z1.append(np.sqrt(-2*np.log(H1[i]))*math.cos(2*(math.pi)*H2[i]))
        z2.append(np.sqrt(-2*np.log(H1[i]))*math.sin(2*(math.pi)*H2[i]))
        
        payoff.append(max(0,S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*z1[i])-X))
        payoff.append(max(0,S0*np.exp((r-0.5*sigma**2)*T+sigma*np.sqrt(T)*z2[i])-X))
    
    price = np.mean(payoff) * np.exp(-r*T)
    
    return price

# Binomial Method

In [6]:
# Eurpean call
def BinomialEurCall(n,u,d,p,S0,X,T):
    E_payoff = 0
    for i in range(0,n+1):
        E_payoff = E_payoff + binom.pmf(i,n,p) * max(0,(S0*(u**i)*(d**(n-i))-X))
    bino_p = E_payoff * np.exp(-r*T) 
    return bino_p    

In [7]:
# American call
def BinomialAmerCall(r,n,T,s0,K,sigma):
    
    # u,d,p
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    
    # create stock price paths dataframe
    x = np.linspace(0, n, n+1)
    mat_u = np.tile(x,(n+1,1))
    mat_d = np.triu(mat_u.T)
    mat_ud = np.triu(mat_u - mat_d)
    p_path = np.triu(s0 * np.power(u,mat_ud) *  np.power(d,mat_d))
    p_path = pd.DataFrame(p_path)
    
    v_path = p_path.copy() # value paths dataframe
    v_path[n] = v_path[n].apply(lambda x: max(0,x-K)) # payoff at T

    for i in np.linspace(n-1,0,n):
        i = int(i)
        for m in np.linspace(0,i,i+1):
            m = int(m)
            EV = max(0,v_path.iloc[m,i]-K)
            CV = (p*v_path.iloc[m,i+1] + (1-p)*v_path.iloc[m+1,i+1])*np.exp(-delta*r)
            if EV > CV:
                v_path.iloc[m,i] = EV
            else:
                v_path.iloc[m,i] = CV

    price = v_path.iloc[0,0]
    
    return price

In [8]:
# American put
def BinomialAmerPut(r,n,T,S0,k,sigma):
    
    # u,d,p
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    
    # create stock price paths dataframe
    x = np.linspace(0, n, n+1)
    mat_u = np.tile(x,(n+1,1))
    mat_d = np.triu(mat_u.T)
    mat_ud = np.triu(mat_u - mat_d)
    p_path = np.triu(S0 * np.power(u,mat_ud) *  np.power(d,mat_d))
    p_path = pd.DataFrame(p_path)
    
    v_path = p_path.copy() # value paths dataframe
    v_path[n] = v_path[n].apply(lambda x: max(0,k-x)) # payoff at maturity

    for i in np.linspace(n-1,0,n):
        i = int(i)
        for m in np.linspace(0,i,i+1):
            m = int(m)
            EV = max(0, k-v_path.iloc[m,i])
            CV = (p*v_path.iloc[m,i+1] + (1-p)*v_path.iloc[m+1,i+1])*np.exp(-delta*r)
            if EV > CV:
                v_path.iloc[m,i] = EV
            else:
                v_path.iloc[m,i] = CV

    price = v_path.iloc[0,0]
    
    return price

In [9]:
# JR binomial method
def BinomialJR(n,sigma,r,S0,X,T):
    delta = T/n
    u = np.exp(delta*(r-0.5*(sigma**2)) + sigma*np.sqrt(delta))
    d = np.exp(delta*(r-0.5*(sigma**2)) - sigma*np.sqrt(delta))
    p = 0.5
    E_payoff = 0
    for i in range(0,n+1):
        E_payoff = E_payoff + binom.pmf(i,n,p) * max(0,(S0*(u**i)*(d**(n-i))-X))
    JR_p = E_payoff * np.exp(-r*T) 
    return JR_p   

# Trinomial method

$$Price = e^{-rT} \sum_{i=0}^n\sum_{k=0}^{n-i}\tbinom{n}{i}p_u^i\tbinom{n-i}{k}p_d^kp_m^{n-k-i}(S_0u^id^k-X)^+
$$

In [10]:
# European Call
def TrinomialEurCall(n,u,d,pu,pm,pd,T,r,sigma,S0,X):
    E_payoff = 0
    for i in range(0,n+1):
        for k in range(0,n-i+1):
            E_payoff = E_payoff + comb(n,i)*comb(n-i,k)*(pu**i)*(pd**k)*(pm**(n-k-i))*max(0,S0*(u**i)*(d**k)-X)
    tri_p = E_payoff * np.exp(-r*T) 
    return tri_p

# LSMC

In [11]:
# function outputs continuation value based on monomials reg
def monomials_reg(dsc_payoff,spot,order):
    
    # initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    # compute X based on order
    for j in range(order):
        X[:,j]=spot**j
    
    # fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [12]:
# function outputs continuation value based on hermite polynomials reg
def hermite_reg(dsc_payoff,spot,order):
    
    # initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    # compute X based on order
    for j in range(order):
        if j==0:
            X[:,j]=spot**j
        elif j==1:
            X[:,j]=spot*2
        elif j==2:
            X[:,j]=4*spot**2-2
        elif j==3:
            X[:,j]=8*spot**3-12*spot
    
    # fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [13]:
def laguerre_reg(dsc_payoff,spot,order):
    
    # initialise object with RHS of reg
    X=np.empty(len(dsc_payoff)*order)
    X.shape=((len(dsc_payoff),order))
    
    # compute X based on order
    for j in range(order):
        if j==0:
            X[:,j]=np.exp(-spot/2)
        elif j==1:
            X[:,j]=np.exp(-spot/2)*(1-spot)
        elif j==2:
            X[:,j]=np.exp(-spot/2)*(1-2*spot+0.5*spot**2)
        elif j==3:
            X[:,j]=np.exp(-spot/2)*(1-3*spot+3*spot+1.5*spot**2-(1/6)*spot**3)
    
    # fit regression
    Exp_Y=sm.OLS(dsc_payoff,X).fit()
    
    return Exp_Y.get_prediction().predicted_mean

In [14]:
# define the generic longstaff-shwartz function
def LSMC(S_0,r,sigma,K,T,n_sim,n_steps,call,poly,order,seed):
    
    # numpy set seed
    np.random.seed(seed)
    
    # compute time-step
    h=T/n_steps
    
    # initialise object containing paths
    S=np.empty(n_sim*n_steps)
    S.shape=((n_sim,n_steps))
    
    # set time-0 value
    S[:,0]=S_0*np.ones(n_sim)
    
    # compute spot paths
    for t in range(1,n_steps):
        Z=np.random.normal(0,1,int(n_sim/2))
        S[:int(n_sim/2),t]=S[:int(n_sim/2),t-1]*np.exp((r-0.5*sigma**2)*h+sigma*np.sqrt(h)*Z)
        S[int(n_sim/2):,t]=S[int(n_sim/2):,t-1]*np.exp((r-0.5*sigma**2)*h-sigma*np.sqrt(h)*Z)
    
    # initialise array for results
    discounted_payoffs=np.zeros(n_sim)
    # start backward induction
    for i in range(len(S[0])-1,0,-1):
        
        # compute payoff 
        if call==1:
            intrinsic_value=np.maximum(S[:,i]-K,0)
            
        else:
            intrinsic_value=np.maximum(K-S[:,i],0)

        # for the final payoff there is no continuation
        if i==len(S[0])-1:

            # discount final payoff to t-1
            discounted_payoffs=intrinsic_value*math.exp(-r*h)

        # compute value from continuation
        else:

            # select paths that are ITM
            ITM_index=intrinsic_value>0
            
            # check if there are enough ITM paths
            if np.count_nonzero(ITM_index)>3:
                
                # discounted future payoff
                Y=discounted_payoffs[ITM_index]
                
                # stock prices
                X=S[ITM_index,i]/S[ITM_index,0]
                
                # perform regression
                if poly==0:
                    # use monomials
                    CV=monomials_reg(Y, X, order)
                elif poly==1:
                    # use hermite polys
                    CV=hermite_reg(Y, X, order)
                else:
                    # use laguerre polys
                    CV=laguerre_reg(Y, X, order)
                    
                # get exercise indices
                exercise_index=CV<intrinsic_value[ITM_index]

                # select appropriate indices to update
                ITM_and_exercised=np.where(ITM_index==True)[0][np.where(exercise_index==True)]

                # check if there are paths where exercise occurs
                if len(ITM_and_exercised)>0:
                    
                    # update discounted payoffs where appropriate
                    discounted_payoffs[ITM_and_exercised]=intrinsic_value[ITM_index][exercise_index]                
                
            # discount payoffs to t-1
            discounted_payoffs=math.exp(-r*h)*discounted_payoffs
            
    return np.mean(discounted_payoffs)

# Finite Difference Method

In [15]:
# Explicit Finite-Difference, European Put
def EFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = delta_t * (sigma**2/(2*(delta_x**2)) + (r-0.5*(sigma**2))/(2*delta_x))
    pm = 1 - delta_t*(sigma**2)/(delta_x**2) - r*delta_t
    pd = delta_t * (sigma**2/(2*(delta_x**2)) - (r-0.5*(sigma**2))/(2*delta_x))
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[N] = np.roll(A[N],N-2)
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        B = np.append(np.zeros(N),(S[N-1,i]-S[N,i])).reshape(N+1,1)
        F = np.dot(A,F) + B
    
    return F

In [16]:
# Implicit Finite-Difference, European Put
def IFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = -0.5 * delta_t * ((sigma**2)/(delta_x**2) + (r-0.5*(sigma**2))/delta_x)
    pm = 1 + delta_t*(sigma**2)/(delta_x**2) + r*delta_t
    pd = -0.5 * delta_t * ((sigma**2)/(delta_x**2) - (r-0.5*(sigma**2))/delta_x)
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[0] = np.append([1,-1],np.zeros(N-1))
    A[N] = np.append(np.zeros(N-1),[1,-1])
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        B = F
        B[0] = 0
        B[N] = S[N,i]-S[N-1,i]
        F = np.dot(inv(A),B)
    
    return F

In [17]:
# Crank-Nicolson Finite-Difference, European Put
def CFD_put(sigma,r,delta_t,delta_x,K,T,S0,SN):
    
    pu = -0.25 * delta_t * (sigma**2/delta_x**2+(r-0.5*(sigma**2))/delta_x)
    pm = 1 + delta_t*(sigma**2)/(2*delta_x**2) + r*delta_t/2
    pd = -0.25 * delta_t * (sigma**2/delta_x**2-(r-0.5*(sigma**2))/delta_x)
    M = round(T/delta_t)
    N = int(round((np.log(SN)-np.log(S0))/delta_x))
    
    X = np.linspace(np.log(SN), np.log(S0), N+1) # ln(S)
    all_S = np.exp(X).reshape(N+1,1) # stock price
    F = np.where(K-all_S>0, K-all_S, 0) # put option values at T
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    P = np.repeat(np.array([pu,pm,pd]),N+1).reshape(3, N+1).T
    A = np.c_[P, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
    for n in range(2,N):
        A[n] = np.roll(A[n],n-1)
    A[0] = np.append([1,-1],np.zeros(N-1))
    A[N] = np.append(np.zeros(N-1),[1,-1])
    
    for i in np.linspace(M-1,0,M):
        i = int(i)
        z = np.repeat(np.array([-pu,-(pm-2),-pd]),N+1).reshape(3, N+1).T
        z = np.c_[z, np.zeros((N-2)*(N+1)).reshape(N+1,N-2)]
        for n in range(2,N):
            z[n] = np.roll(z[n],n-1)
        Z = np.dot(z,F)
        Z[0] = 0
        Z[N] = S[N,i]-S[N-1,i]
        F = np.dot(inv(A),Z)
    
    return F

In [18]:
# Generalization of Finite-Difference Schemes, Americna call & put
def GFD_American(sigma,r,delta_t,delta_S,K,T,S0,SN,alpha,c_p):
    # c_p = 1 means call option, c_p = -1 means put option
    
    M = round(T/delta_t)
    N = int((SN-S0)/delta_S)
    all_S = np.linspace(SN,S0,N+1)
    j = all_S/delta_S
    ex_values = np.where(c_p*(all_S-K)>0, c_p*(all_S-K), 0).reshape(N+1,1)
    S = np.repeat(all_S,M+1).reshape(N+1,M+1)
    
    # calculate a1,a2,a3,b1,b2,b3
    A = np.zeros((N+1)**2).reshape(N+1,N+1)
    B = np.zeros((N+1)**2).reshape(N+1,N+1)
    A[0,0] = 1
    A[0,1] = -1
    A[N,N-1] = 1
    A[N,N] = -1
    for n in range(1,N):
        a1 = (1-alpha) * ((sigma**2)*(j[n]**2)-r*j[n]) / 2
        a2 = -1/delta_t - ((sigma**2)*(j[n]**2)+r) * (1-alpha)
        a3 = (1-alpha) * ((sigma**2)*(j[n]**2)+r*j[n]) / 2
        b1 = alpha * ((sigma**2)*(j[n]**2)-r*j[n]) / 2
        b2 = 1/delta_t - ((sigma**2)*(j[n]**2)+r) * alpha
        b3 = alpha * ((sigma**2)*(j[n]**2)+r*j[n]) / 2
        A[n,n-1] = a3
        A[n,n] = a2
        A[n,n+1] = a1
        B[n,n-1] = -b3
        B[n,n] = -b2
        B[n,n+1] = -b1
    
    F = ex_values
    for i in np.linspace(M-1,0,M):
        i = int(i)
        d = np.dot(B,F)
        if c_p == 1:
            d[0] = delta_S
            d[N] = 0
        if c_p == -1:
            d[0] = 0
            d[N] = delta_S
        F = np.dot(inv(A),d)
        F = np.where(F >= ex_values, F, ex_values)
        
    return F